In [15]:
import findspark
findspark.init()
import pyspark # only run after findspark.init()

from pyspark.sql import SparkSession
from pyspark.context import SparkContext 
spark = SparkSession.builder.getOrCreate()#create spark session 
sc = spark.sparkContext#create sparkContext
from pyspark.sql.types import  (StructType, 
                                StructField, 
                                DateType, 
                                BooleanType,
                                DoubleType,
                                IntegerType,
                                StringType,
                                DecimalType,
                                LongType,
                                ArrayType,
                                TimestampType)
sample_schema = StructType([StructField("id",  IntegerType(), True),
                            StructField("timeAtServer", DoubleType(), True),
                            StructField("aircraft",  IntegerType(), True ),
                            StructField("latitude", DoubleType(), True),
                            StructField("longitude", DoubleType(), True  ),
                            StructField("baroAltitude", DoubleType(), True ),
                            StructField("geoAltitude", DoubleType(), True ),
                            StructField("numMeasurements",IntegerType(), True),
                            StructField("measurements", StringType(), True)           
                            ])
sample_aircarft = spark.read.csv("C:/Users/rzouga/Desktop/PFE/TEST/sample.csv",
                       header = True, 
                        schema = sample_schema)
#sort our data  by distinct aircraft and time at server 
sorted_flights =sample_aircarft.orderBy('aircraft','timeAtServer')
#exapmle of a flight trip 296
trip296=sorted_flights.filter('aircraft= 296')
trip296=trip296.select('latitude','longitude','geoAltitude','timeAtServer')
trip296.show(5)

+----------------+----------------+-----------+-----------------+
|        latitude|       longitude|geoAltitude|     timeAtServer|
+----------------+----------------+-----------+-----------------+
|50.4569929737156|7.42067079286318|   10812.78|0.425000190734863|
|50.4567260742188|7.42263793945312|   10812.78|0.942000150680542|
|50.4564343468618|7.42460508604308|   10812.78| 1.50300002098084|
|50.4562946901483|7.42586702913851|   10812.78| 1.94500017166138|
|50.4558563232422|7.42914300215872|   10812.78| 3.00600004196167|
+----------------+----------------+-----------+-----------------+
only showing top 5 rows



In [16]:
from pyspark import SparkContext
from pyspark.sql import SQLContext
from pyspark.sql import functions as F
from pyspark.sql.window import Window
my_window = Window.partitionBy().orderBy("timeAtServer")
trip296= trip296.withColumn("lat_prev", F.lag(trip296.latitude).over(my_window))
trip296= trip296.withColumn("long_prev", F.lag(trip296.longitude).over(my_window))
trip296= trip296.withColumn("T_prev", F.lag(trip296.timeAtServer).over(my_window))
trip296.show(5)

+----------------+----------------+-----------+-----------------+----------------+----------------+-----------------+
|        latitude|       longitude|geoAltitude|     timeAtServer|        lat_prev|       long_prev|           T_prev|
+----------------+----------------+-----------+-----------------+----------------+----------------+-----------------+
|50.4569929737156|7.42067079286318|   10812.78|0.425000190734863|            null|            null|             null|
|50.4567260742188|7.42263793945312|   10812.78|0.942000150680542|50.4569929737156|7.42067079286318|0.425000190734863|
|50.4564343468618|7.42460508604308|   10812.78| 1.50300002098084|50.4567260742188|7.42263793945312|0.942000150680542|
|50.4562946901483|7.42586702913851|   10812.78| 1.94500017166138|50.4564343468618|7.42460508604308| 1.50300002098084|
|50.4558563232422|7.42914300215872|   10812.78| 3.00600004196167|50.4562946901483|7.42586702913851| 1.94500017166138|
+----------------+----------------+-----------+---------

In [24]:
'''NAVIGATION ON ESTIMATE
 Two cases will arise:
 - the aircraft speed is imposed
- The arrival time is imposed
THE ELEMENTS OF NAVIGATION ON ESTIMATE:
- Starting;
- The road joining the two points;
- the speed of the airplane.
- timeatserver.
'''
#Let's calculate distance between each distance between each reported point and cumulative distance .
#calcul Velocity of our aircraft V,  Bearing 
'''Formula to Find Bearing or Heading angle between two points: Latitude Longitude
Formula to Find Bearing or Heading angle between two points. 
Bearing or heading angle is used to define navigation generally in the field of aircraft or marine
or Vehicle navigation or while working for land surveying. So what’s the heading or bearing?
How we can find bearing between the two points on earth, with the formula? 
Or How we can find the other point, when one point, actual traveled distance and bearing is given?
Bearing can be defined as direction or an angle, between the north-south line of earth or meridian 
and the line connecting the target and the reference point. 
While Heading is an angle or direction where you are currently navigating in. 
This means to reach a particular destination you need to adjust your heading direction with the bearing. 
Generally a ‘compass’ is an instrument, which gives you the direction information for navigation.
Let us look on formula and tool for bearing:

    Let ‘R’ be the radius of Earth,
    ‘L’ be the longitude,
    ‘θ’ be latitude,
    ‘β‘ be Bearing.
    
Denote point A and B as two different points, where ‘La’ is point A longitude and ‘θa’ is point A latitude,
similarly assume for point B. 
Bearing would be measured from North direction i.e 0° bearing means North, 90° bearing is East,
180° bearing is measured to be South, and 270° to be West.

Note: If bearing is denoted with +ve or –ve initials whose values lies between 0° to 180°,
then –ve is denoted for South and West sides.
Formula to find Bearing, when two different points latitude, longitude is given:

Bearing from point A to B, can be calculated as,

β = atan2(X,Y),

where, X and Y are two quantities and can be calculated as:

X = cos θb * sin ∆L

Y = cos θa * sin θb – sin θa * cos θb * cos ∆L'''
import geohash2
import pyspark.sql.functions as F
from pyspark.sql.functions import udf,col
from pyspark.sql.types import IntegerType
from pyspark.sql.types import *
from pyspark.sql.functions import udf
import pandas as pd
from math import radians, cos, sin, asin, sqrt
from geopy.distance import geodesic
from geopy.geocoders import Nominatim
from math import radians, cos, sin, asin, sqrt, atan2, degrees


def get_distance_2d(lat_1, lng_1, lat_2, lng_2): 
    A =(lat_1, lng_1)
    B = (lat_2, lng_2)
    if lat_1==None and lng_1==None:
        d=0
    else:
        d=geodesic(A, B).kilometers
    return d
def get_distance_3d(distance_2d, alt1, alt2): 
    distance_3d = sqrt(distance_2d**2+(alt2-alt1)**2)
    return  distance_3d
def get_Difference( alt1, alt2): 
    Difference= alt1-alt2
    return Difference
udf_get_distance_2d = F.udf(get_distance_2d)
udf_get_distance_3d = F.udf(get_distance_3d)
udf_get_Difference = F.udf(get_Difference)
def initial_bearing(lat1,long1,lat2,long2):
    if lat1==None and long1==None:
        compass_bearing=0
    else:
        lat1 = radians(float(lat1))
        lat2 = radians(float(lat2))

        diffLong = radians(float(long2) - float(long1))

        x = sin(diffLong) * cos(lat2)
        y = cos(lat1) * sin(lat2) - (sin(lat1)
            * cos(lat2) * cos(diffLong))

        initial_bearing = atan2(x, y)

         # Now we have the initial bearing but math.atan2 return values
         # from -180° to + 180° which is not what we want for a compass bearing
         # The solution is to normalize the initial bearing as shown below
        initial_bearing = degrees(initial_bearing)
        compass_bearing = (initial_bearing + 360) % 360

    return compass_bearing 
udf_initial_bearing = F.udf(initial_bearing)
trip296=trip296.withColumn('bearing', udf_initial_bearing('lat_prev','long_prev','latitude','longitude').cast(DoubleType()))
trip296=trip296.withColumn('D_delta', udf_get_distance_2d('lat_prev','long_prev','latitude','longitude').cast(DoubleType()))
trip296.D_delta.cast(DoubleType())
trip296.bearing.cast(DoubleType())
trip296=trip296.withColumn('D_cumu',F.sum('D_delta').over(my_window))
trip296=trip296.withColumn('t_delta',trip296.timeAtServer-trip296.T_prev)
trip296=trip296.withColumn('t_delta',trip296.timeAtServer-trip296.T_prev)
#lets calculate velocity km per hour km/h ,of our aircraft :
trip296=trip296.withColumn('V',(trip296.D_delta*3600)/trip296.t_delta)
trip296= trip296.withColumn("V_prev", F.lag(trip296.V).over(my_window))
trip296.select('D_delta','D_cumu','t_delta','V','V_prev','bearing').show()

+-------------------+-------------------+------------------+------------------+------------------+------------------+
|            D_delta|             D_cumu|           t_delta|                 V|            V_prev|           bearing|
+-------------------+-------------------+------------------+------------------+------------------+------------------+
|                0.0|                0.0|              null|              null|              null|               0.0|
|0.14281513356561373|0.14281513356561373|0.5169999599456789| 994.4574867863229|              null| 102.0295850933378|
|0.14341552037181954| 0.2862306539374333|0.5609998703002981| 920.3137124830027| 994.4574867863229|103.11150724382543|
|0.09095317108040109| 0.3771838250178344|  0.44200015068054|  740.794806031861| 920.3137124830027| 99.86038738012792|
|0.23769947506939193| 0.6148833000872262|1.0609998703002899| 806.5204664046009|  740.794806031861|101.86817965411535|
|0.14456135920372912| 0.7594446592909554|0.5690000057220

In [25]:
t=trip296.select('bearing','t_delta','V').describe()
t.show()

+-------+------------------+------------------+------------------+
|summary|           bearing|           t_delta|                 V|
+-------+------------------+------------------+------------------+
|  count|                32|                31|                31|
|   mean|  98.6028805140275|0.5556129024874786| 850.3749879796335|
| stddev|  18.0179319722314|0.1824343737786809| 136.3494609410782|
|    min|               0.0|0.3689999580382999|  642.972904833689|
|    max|103.11150724382543|1.0609998703002899|1112.0194593707554|
+-------+------------------+------------------+------------------+



In [26]:

'''for our porject let's   find a lat lon point, when bearing, distance and another lat lon is given


Let us assume a condition, where you want to find out the where will an Airplane will land up, 
if you have following details of that Airplane, i.e actual distance it will travel, 
the bearing and the starting point (lat,lon)?

    Let first point latitude be la1,
    longitude as lo1,
    d be distance,
    R as radius of Earth,
    Ad be the angular distance i.e d/R and
    θ be the bearing,

Here is the formula to find the second point, when first point, bearing and distance is known:

    latitude of second point = la2 =  asin(sin la1 * cos Ad  + cos la1 * sin Ad * cos θ), and
    longitude  of second point = lo2 = lo1 + atan2(sin θ * sin Ad * cos la1 , cos Ad – sin la1 * sin la2)

'''
# if we dont have  distance , let's calculat it d= t_delta*V 
#first point  la1=lat_prev lon1=long_prev
def getEndlat2(lat1,lon1,bearing,d):
    if lat1==None and long1==None:
        lat2=None
    else:
        R = 6378.1                   #Radius of the Earth
        brng = math.radians(bearing) #convert degrees to radians
         #distance to km
        lat2 = math.asin( math.sin(lat1)*math.cos(d/R) + math.cos(lat1)*math.sin(d/R)*math.cos(brng))
       
        return lat2
def getEndlon2(lat1,lon1,bearing,d):
    if lat1==None and long1==None:
        long2=None
    else:
        R = 6378.1                   #Radius of the Earth
        brng = math.radians(bearing) #convert degrees to radians
         #distance to km
        lat2 = math.asin( math.sin(lat1)*math.cos(d/R) + math.cos(lat1)*math.sin(d/R)*math.cos(brng))
        lon2 = lon1 + math.atan2(math.sin(brng)*math.sin(d/R)*math.cos(lat1),math.cos(d/R)-math.sin(lat1)*math.sin(lat2))
       
        return  lon2
# or we can use more accurate calcul by geographiclib
from geographiclib.geodesic import Geodesic
import math
geod = Geodesic.WGS84  # define the WGS84 ellipsoid
g = geod.Direct(-41.294444, 174.814444, 1.02749621782165, 80*1000)
print(g['lat2'],g['lon2'])

(0.44119858771455633, 174.8141952645693)

In [32]:
#let's calculate d= t_delta*V, we have it D_delta
#first point  la1=lat_prev lon1=long_prev
#estimate our point b not detected by the ADS-B
import geopy
from geopy.distance import geodesic
from geographiclib.geodesic import Geodesic
import math
geod = Geodesic.WGS84  # define the WGS84 ellipsoid
# given: lat1, lon1, b = bearing in degrees, d = distance in kilometers
def getEndlat2(lat1,lon1,bearing,d):
    if lat1==None and lon1==None:
        lat2=None
    else:
        geod = Geodesic.WGS84  # define the WGS84 ellipsoid
        g = geod.Direct(lat1, lon1, bearing, d*1000)
        lat2 = g['lat2']
        return lat2

def getEndlon2(lat1,lon1,bearing,d):
    if lat1==None and lon1==None:
        lon2=None
    else:
        geod = Geodesic.WGS84  # define the WGS84 ellipsoid
        geod = Geodesic.WGS84
        g = geod.Direct(lat1, lon1, bearing, d*1000)
        lon2 = g['lon2']
        return lon2
udf_getEndlat2 = F.udf(getEndlat2)
udf_getEndlon2= F.udf(getEndlon2)
trip296=trip296.withColumn('lat2', udf_getEndlat2('lat_prev','long_prev','bearing','D_delta'))
trip296=trip296.withColumn('lon2', udf_getEndlon2('lat_prev','long_prev','bearing','D_delta'))
trip296.select('latitude','longitude','lat2','lon2').show()

+----------------+----------------+------------------+------------------+
|        latitude|       longitude|              lat2|              lon2|
+----------------+----------------+------------------+------------------+
|50.4569929737156|7.42067079286318|              null|              null|
|50.4567260742188|7.42263793945312| 50.45672537675061| 7.422637706926938|
|50.4564343468618|7.42460508604308| 50.45643359088953| 7.424604810572474|
|50.4562946901483|7.42586702913851| 50.45629431980791| 7.425866928434988|
|50.4558563232422|7.42914300215872| 50.45585517629454|7.4291426250369685|
| 50.455596406581|7.43113749736064|50.455595725035856| 7.431137279131482|
|50.4554101976298|7.43239944045608| 50.45540971482283| 7.432399265417156|
|50.4551696777344| 7.4344193307977| 50.45516904282963| 7.434419145041252|
|50.4548981230138|7.43633373363598| 50.45489741617562|  7.43633348729033|
|50.4547119140625|7.43766990867821| 50.45471142857068| 7.437669742448551|
|50.4544830322266|7.43962338096217| 50

In [36]:
#calculat erreur distance 
trip296=trip296.withColumn('D_erreur', udf_get_distance_2d('latitude','longitude','lat2','lon2').cast(DoubleType()))
trip296.select('latitude','longitude','lat2','lon2','D_erreur').show()
                           

+----------------+----------------+------------------+------------------+--------------------+
|        latitude|       longitude|              lat2|              lon2|            D_erreur|
+----------------+----------------+------------------+------------------+--------------------+
|50.4569929737156|7.42067079286318|              null|              null|   5635.912043657477|
|50.4567260742188|7.42263793945312| 50.45672537675061| 7.422637706926938|7.932262969050363E-5|
|50.4564343468618|7.42460508604308| 50.45643359088953| 7.424604810572474|8.633815385949807E-5|
|50.4562946901483|7.42586702913851| 50.45629431980791| 7.425866928434988|4.181199173931333E-5|
|50.4558563232422|7.42914300215872| 50.45585517629454|7.4291426250369685|1.303645693021591...|
| 50.455596406581|7.43113749736064|50.455595725035856| 7.431137279131482|7.738141143246107E-5|
|50.4554101976298|7.43239944045608| 50.45540971482283| 7.432399265417156|5.512619319471214E-5|
|50.4551696777344| 7.4344193307977| 50.45516904282

In [37]:
#using Pandas power to analyse our filtred data
import pandas as pd
trip296_pandas_DataFrame = pd.DataFrame.from_records(trip296.collect(), columns=trip296.columns)
trip296_pandas_DataFrame.head(20)

,latitude,longitude,geoAltitude,timeAtServer,lat_prev,long_prev,T_prev,bearing,D_delta,D_cumu,t_delta,V,V_prev,lat2,lon2,D_erreur
0,50.456993,7.420671,10812.78,0.425,NaN,NaN,NaN,0.000000,0.000000,0.000000,NaN,NaN,NaN,None,None,5635.912044
1,50.456726,7.422638,10812.78,0.942,50.456993,7.420671,0.425,102.029585,0.142815,0.142815,0.517,994.457487,NaN,50.45672537675061,7.422637706926938,0.000079
2,50.456434,7.424605,10812.78,1.503,50.456726,7.422638,0.942,103.111507,0.143416,0.286231,0.561,920.313712,994.457487,50.45643359088953,7.424604810572474,0.000086
3,50.456295,7.425867,10812.78,1.945,50.456434,7.424605,1.503,99.860387,0.090953,0.377184,0.442,740.794806,920.313712,50.45629431980791,7.425866928434988,0.000042
4,50.455856,7.429143,10812.78,3.006,50.456295,7.425867,1.945,101.868180,0.237699,0.614883,1.061,806.520466,740.794806,50.45585517629454,7.4291426250369685,0.000130
5,50.455596,7.431137,10812.78,3.575,50.455856,7.429143,3.006,101.566979,0.144561,0.759445,0.569,914.623705,806.520466,50.455595725035856,7.431137279131482,0.000077
6,50.455410,7.432399,10812.78,4.090,50.455596,7.431137,3.575,103.048101,0.091981,0.851426,0.515,642.972905,914.623705,50.45540971482283,7.432399265417156,0.000055
7,50.455170,7.434419,10812.78,4.617,50.455410,7.432399,4.090,100.592636,0.145919,0.997345,0.527,996.791082,642.972905,50.45516904282963,7.434419145041252,0.000072
8,50.454898,7.436334,10812.78,5.134,50.455170,7.434419,4.617,102.559197,0.139270,1.136615,0.517,969.772548,996.791082,50.45489741617562,7.43633348729033,0.000081
9,50.454712,7.437670,10812.78,5.625,50.454898,7.436334,5.134,102.345828,0.097126,1.233740,0.491,712.123636,969.772548,50.45471142857068,7.437669742448551,0.000055


In [42]:
#plot aircraft location aircraft icon + Estimated aircraft blue marker .
import folium
from folium.plugins import FloatImage
from folium import plugins
# generate a new map
folium_map = folium.Map()
trip296_pandas_DataFrame['latitude'] = pd.to_numeric(trip296_pandas_DataFrame['latitude'],errors='coerce')
trip296_pandas_DataFrame['longitude'] = pd.to_numeric(trip296_pandas_DataFrame['longitude'],errors='coerce')
trip296_pandas_DataFrame['latitude'] = trip296_pandas_DataFrame['latitude'].astype(float)
trip296_pandas_DataFrame['longitude'] = trip296_pandas_DataFrame['longitude'].astype(float)

trip296_pandas_DataFrame= trip296_pandas_DataFrame.dropna()

for index, row in  trip296_pandas_DataFrame.iterrows():
    folium.Marker([float(row['latitude']),float( row['longitude'])],
         popup = ('bearing: ' + str(row['bearing'] ).capitalize() + '<br>'
                 'Distance_cumul: ' + str(row['D_cumu']) + '<br>'
                  'Velocity: ' + str(row['V']) + '<br>'
                 
                ),
         icon=folium.features.CustomIcon('http://icons.iconarchive.com/icons/google/noto-emoji-travel-places/256/42586-airplane-icon.png',icon_size=(20, 20))
         ).add_to(folium_map)


for index, row in  trip296_pandas_DataFrame.iterrows():
    folium.Marker([float(row['lat2']),float( row['lon2'])],
         popup = ('D_erreur: ' + str(row['D_erreur'] ).capitalize() + '<br>'
                 'timeAtServer:' + str(row['timeAtServer']) + '<br>'
                  'geoAltitude: ' + str(row['geoAltitude']) + '<br>'
                 
                ),
        
         ).add_to(folium_map)


folium_map
